# 플라스크 임포트  

In [1]:
!pip install PyMySQL

In [2]:
from flask import Flask
from flask import redirect, request , jsonify
from flask_cors import CORS
import pandas as pd
import pickle
from konlpy.tag import Okt, Kkma
okt = Okt()
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pymysql

# PymySQL

In [3]:
import pymysql

In [4]:
db = pymysql.connect(host = 'project-db-stu.ddns.net', port = 3307, user= 'dodam',
                    passwd='115', db='dodam', charset='utf8')
db

In [5]:
cursor = db.cursor(pymysql.cursors.DictCursor)
cursor

In [6]:
sql = "SELECT * FROM `t_diary`;"
cursor.execute(sql)
result = cursor.fetchall()
result

[{'d_seq': 1,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 즐거운 스마트 인재 개발원 ',
  'd_content': '오늘은 점심은 또 뭐먹을까 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 15)},
 {'d_seq': 2,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 오늘도 즐거운 스마트 인재 개발원 ',
  'd_content': '오늘은 점심은 짜장이다! ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 16)},
 {'d_seq': 3,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 매일 즐거운 스마트 인재 개발원 ',
  'd_content': '오늘은 점심은 돈가스다! ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 17)},
 {'d_seq': 4,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '슬픈 토요일 등원 ',
  'd_content': '오늘은 점심은 맛있는거 먹어야지 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 18)},
 {'d_seq': 5,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '즐거운 야근 너무 즐거워 ',
  'd_content': '에너지 드링크가 필요해 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 19)}]

In [7]:
def myTokenizer2(text):
    d = pd.DataFrame(okt.pos(text, norm=True, stem=True), columns=['morph','tag'])
    d.set_index('tag', inplace=True)
    if ('Noun' in d.index) | ('Verb' in d.index) | ('Adjective' in d.index):
        labels = ['Noun','Verb','Adjective']
        return d.loc[d.index.intersection(labels)].dropna()['morph'].values
    else :
        return []

In [8]:
def height_predict(model, input_height, n=72):
    max_len = 72
    for _ in range(len(input_height),n):# n번 반복
        encoded = pad_sequences(input_height, maxlen=max_len-1, padding='pre',dtype='float32').reshape(71,-1)
        result = model.predict([encoded.tolist()],verbose=0)
        input_height = np.hstack([input_height,result])
    return input_height

In [ ]:
# 수정
result = {}
app = Flask(__name__)
CORS(app)

# with open('dodam_model.pkl', 'rb') as f:
#     model = pickle.load(f)
    
@app.route("/post2",  methods=["POST", "GET"])

    
def test():
    if (request.method=="POST"):
       
        display(request.form)
        m_id=request.form.get('m_id')
        c_seq=request.form.get('c_seq')
        d_title=request.form.get('d_title')
        d_content=request.form.get('d_content')
        d_msg=request.form.get('d_msg')
        d_date=request.form.get('d_date')
        
        sql = '''insert into `t_diary` (m_id, c_seq, d_title, d_content, d_msg, d_date) 
values(%s, %s, %s, %s, %s, %s)'''
        cursor.execute(sql , (m_id, c_seq, d_title, d_content, d_msg, d_date))
        db.commit()
        result = cursor.fetchall()
        result
        #텍스트마이닝 
        result = request.form.get('d_content')
        with open('dodam_cv.pkl', 'rb') as f:
    # 정의된 모델 선언
            cv = pickle.load(f)
    # 사용자가 입력한 데이터를 모델이 읽을 수 있는 데이터로 변환
            data_review = cv.transform([result])
    # 예측 결과를 a에 담는다.
            with open('dodam_lr.pkl','rb') as f1:
                lr_model = pickle.load(f1)
                pre = lr_model.predict_proba(data_review)
                feel_list = ['기쁨','분노','불안','슬픔']
                result_list = []
                for i in range(len(feel_list)):
                    result_list.append(pre[0][i])
                    print(f"{feel_list[i]} : {pre[0][i]}")
                e_joy = round((result_list[0]*100),2)
                e_joy = e_joy.item()
                e_sorrow = round((result_list[3]*100),2)
                e_sorrow = e_sorrow.item()
                print(e_sorrow)
                print(type(e_sorrow))
                e_anger = round((result_list[1]*100),2)
                e_anger = e_anger.item()
                e_unrest = round((result_list[2]*100),2)
                e_unrest = e_unrest.item()
                sql1 = '''insert into `t_emotion` (m_id, e_joy, e_sorrow, e_anger, e_unrest, d_date) 
values(%s, %s, %s, %s, %s, %s)'''
                cursor.execute(sql1, (m_id, e_joy, e_sorrow, e_anger, e_unrest, d_date))
                db.commit()
                result1 = cursor.fetchall()
                result1
                print(pre[0])
#         result = test()
#         print(result)
        result2 = str("%.2f%%"%(pre[0][0]*100)) + "<br>" + str(feel_list[1])+" : "+str("%.2f%%"%(pre[0][1]*100)) + "<br>"+str(feel_list[2])+" : "+str("%.2f%%"%(pre[0][2]*100)) + "<br>"+str(feel_list[3])+" : "+str("%.2f%%"%(pre[0][3]*100))
        #return str(feel_list[0])+" : "
        return redirect("http://localhost:8089/dodam/diary.do?result2="+result2)
    
    else:
        
        return "NOT FOUND 404"
    
#키 예측 모델 
@app.route("/height",  methods=["POST", "GET"])
def test2():
    if (request.method=="POST"):

        model = load_model('Height.h5')   
        display(request.form)
        input_height = request.form.get('height')
        print(input_height)
        result = height_predict(model, np.array([[input_height]]))
        print(result)
        tr = str(float(result[0][71])*100//1/100)
        return redirect("http://localhost:8089/dodam/note.do?tr="+tr)
    else:
        return "NOT FOUND 404"

    
if __name__ == "__main__":
    app.run(host = "127.0.0.1", port =7000)
    # result = test()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:7000/ (Press CTRL+C to quit)


ImmutableMultiDict([('height', '52')])

52


127.0.0.1 - - [21/Apr/2022 16:53:00] "POST /height HTTP/1.1" 302 -


...

0.0


127.0.0.1 - - [21/Apr/2022 16:58:19] "POST /height HTTP/1.1" 302 -


[['0.0' '23.777151' '33.947655' '35.476326' '37.031174' '38.8982'
  '41.6095' '44.449215' '46.483784' '48.32634' '50.110477' '51.775867'
  '53.29609' '54.698215' '56.00648' '57.23103' '58.3839' '59.478737'
  '60.529022' '61.564465' '62.64005' '63.781536' '64.94435' '66.075485'
  '67.16012' '68.20527' '69.22193' '70.2216' '71.215096' '72.21012'
  '73.209114' '74.20793' '75.19622' '76.159615' '77.08333' '77.95582'
  '78.77109' '79.52893' '80.23366' '80.89237' '81.51337' '82.10518'
  '82.67604' '83.23365' '83.785065' '84.336685' '84.89428' '85.46298'
  '86.04742' '86.65188' '87.28082' '87.93952' '88.63496' '89.37714'
  '90.18121' '91.07' '92.071915' '93.20821' '94.48346' '95.89697'
  '97.453026' '99.14888' '100.95451' '102.805176' '104.62033' '106.33825'
  '107.9361' '109.4237' '110.82898' '112.18902' '113.54067' '114.899376']]


127.0.0.1 - - [21/Apr/2022 16:58:21] "POST /height HTTP/1.1" 302 -


[['0.0' '23.777151' '33.947655' '35.476326' '37.031174' '38.8982'
  '41.6095' '44.449215' '46.483784' '48.32634' '50.110477' '51.775867'
  '53.29609' '54.698215' '56.00648' '57.23103' '58.3839' '59.478737'
  '60.529022' '61.564465' '62.64005' '63.781536' '64.94435' '66.075485'
  '67.16012' '68.20527' '69.22193' '70.2216' '71.215096' '72.21012'
  '73.209114' '74.20793' '75.19622' '76.159615' '77.08333' '77.95582'
  '78.77109' '79.52893' '80.23366' '80.89237' '81.51337' '82.10518'
  '82.67604' '83.23365' '83.785065' '84.336685' '84.89428' '85.46298'
  '86.04742' '86.65188' '87.28082' '87.93952' '88.63496' '89.37714'
  '90.18121' '91.07' '92.071915' '93.20821' '94.48346' '95.89697'
  '97.453026' '99.14888' '100.95451' '102.805176' '104.62033' '106.33825'
  '107.9361' '109.4237' '110.82898' '112.18902' '113.54067' '114.899376']]
